In [87]:
import numpy as np
import os, os.path
import matplotlib.image as matlib

In [88]:
plane = "sagittal"
mri_slice = 99
mouse_x = 212
mouse_y = 212

In [89]:
npy_file = "../react-ui/src/assets/P57-16/mri/indices_" + plane + "/slice_0" + str(mri_slice) + ".npy"

In [92]:
npy_slice_array = np.load(npy_file)

print(np.isfortran(npy_slice_array))
print(npy_slice_array.shape)
#print(npy_slice_array[45])

True
(224, 448)


In [93]:
# get the block number from the numpy array
block = npy_slice_array[mouse_x][mouse_y]
print(block)

0


In [35]:
# define a vector to be used later (to get the histology coords)
# an earlier version of this notebook did +1 to the mri_slice here, need to check if this is necessary.
mri_vector=np.array([mouse_x, mouse_y, mri_slice, 1.0])

In [36]:
# open the matrix txt file
matrix_txt = open("../react-ui/src/assets/P57-16/mri/matrices/block_" + str(block) + ".txt")
read_matrix_txt = matrix_txt.read()

In [37]:
# parse the text file and convert it to an array
matrix = read_matrix_txt.replace('\n', ' ')
matrix = matrix.split(" ")
matrix = matrix[:-1] # pops the last element in the array

In [38]:
# convert the array of strings to an array of floats
matrix_as_floats = []
for item in matrix:
    matrix_as_floats.append(float(item))

In [39]:
# divide matrix array into chuncks to form a multidimensional array
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]

# how many elements should have
n = 4
  
chunked_array = list(divide_chunks(matrix_as_floats, n))
print(chunked_array)

[[3.9918815921927324, 0.14203936785714244, 0.2353534993986506, -95.3126553016657], [0.16442745659295432, -1.3162776492428165, -3.803865995716457, 969.8247167526811], [-0.0112355845778312, 0.55207262125821, -0.23935829358640404, -4.480988935563403], [0.0, 0.0, 0.0, 1.0]]


In [40]:
transform = np.array(chunked_array)
# print(transform)

In [41]:
blockvector = transform.dot(mri_vector)

In [42]:
print(blockvector)

[156.26655979 291.6532176    9.52021904   1.        ]


In [ ]:
# test_transform = np.array([[0.31385434307807203, -3.664457652247729, 1.6199030926177134, 666.0080271821023],
# [-0.7744411837137424, 1.4870493065731998, 3.6535143658756977, -139.31333561581766],
# [-1.2110952755381341, -0.18593341100766062, -0.1914204859080043, 118.27995513465419],
# [0.0, 0.0, 0.0, 1.0]])

In [11]:
histology = "../react-ui/src/assets/P57-16/histology"


In [12]:
folder = os.path.join(histology,"25")
slice_folder = os.path.join(folder,"slices_HE")
if os.path.exists(slice_folder):
    print(slice_folder)
else:
    print("the folder does not exist")
images = [name for name in os.listdir(slice_folder) if os.path.isfile(os.path.join(slice_folder, name))]
print(len(images))
image = None
if len(images) > 0:
    image = images[0]

../react-ui/src/assets/P57-16/histology/25/slices_HE
38


In [13]:
if not image == None:
    fullPath = os.path.join(slice_folder, image)
    readImage = matlib.imread(fullPath)
    print(readImage.shape)

(805, 552, 3)
